In [ ]:
import torch
from fl_g13.editing.sparseSGDM import SparseSGDM
from torch.nn import CrossEntropyLoss
from torch.optim.lr_scheduler import CosineAnnealingLR

import flwr
from flwr.simulation import run_simulation
from fl_g13.architectures import BaseDino
from fl_g13.fl_pytorch.client_app import get_client_app
from fl_g13.fl_pytorch.server_app import get_server_app

print(f"Flower {flwr.__version__} / PyTorch {torch.__version__}")

2025-05-09 17:33:50.321 | INFO     | fl_g13.config:<module>:11 - PROJ_ROOT path is: /home/massimiliano/Projects/fl-g13


In [ ]:
import os
import urllib.request

# TODO: move in a "make client dependencies function"
def download_if_not_exists(file_path: str, file_url: str):
    """
    Checks if a file exists at the given path. If it does not, downloads it from the specified URL.

    Parameters:
    - file_path (str): The local path to check and save the file.
    - file_url (str): The URL from which to download the file.
    """
    if not os.path.exists(file_path):
        print(f"'{file_path}' not found. Downloading from {file_url}...")
        try:
            urllib.request.urlretrieve(file_url, file_path)
            print("Download complete.")
        except Exception as e:
            print(f"Failed to download file: {e}")
    else:
        print(f"'{file_path}' already exists.")

download_if_not_exists(
    "vision_transformer.py",
    "https://raw.githubusercontent.com/facebookresearch/dino/refs/heads/main/vision_transformer.py"
)

download_if_not_exists(
    "utils.py",
    "https://raw.githubusercontent.com/facebookresearch/dino/refs/heads/main/utils.py"
)

'vision_transformer.py' already exists.
'utils.py' already exists.


In [ ]:
# Settings
CHECKPOINT_DIR = "/home/massimiliano/Projects/fl-g13/checkpoints"

# Model hyper-parameters
model_class = BaseDino
head_layers=3
head_hidden_size=512
dropout_rate=0.0
unfreeze_blocks=1

# Training hyper-parameters
starting_lr = 1e-3
momentum = 0.9
weight_decay=1e-5
T_max=8
eta_min=1e-5

# Federated Training setting
batch_size = 64
local_epochs = 2
number_of_rounds = 2
fraction_fit = 1
fraction_evaluate = 0.1
number_of_clients = 3
min_num_clients = 3
partition_type = "iid" # or "shard"
num_shards_per_partition = 6
use_wandb = False

# Device settings
device = "cuda" if torch.cuda.is_available() else "cpu"
backend_config = {"client_resources": {"num_cpus": 1, "num_gpus": 0.0}}

# When running on GPU, assign an entire GPU for each client
if device == "cuda":
    backend_config["client_resources"] = {"num_cpus": 1, "num_gpus": 1}

    # Refer to our Flower framework documentation for more details about Flower simulations
    # and how to set up the `backend_config`

print(f"Training on {device}")

Training on cuda
Flower 1.17.0 / PyTorch 2.6.0+cu124


In [ ]:
# Model
model = model_class(
    head_layers=head_layers, 
    head_hidden_size=head_hidden_size, 
    dropout_rate=dropout_rate, 
    unfreeze_blocks=unfreeze_blocks
    )
model.to(device)

mask = [torch.ones_like(p, device=p.device) for p in model.parameters()] # Must be done AFTER the model is moved to CUDA
optimizer = SparseSGDM(
    model.parameters(),
    mask=mask,
    lr=starting_lr,
    momentum=momentum,
    weight_decay=weight_decay
    )
scheduler = CosineAnnealingLR(
    optimizer=optimizer, 
    T_max=T_max, 
    eta_min=eta_min
    )
criterion = CrossEntropyLoss()

client_app = get_client_app(
    model=model, 
    optimizer=optimizer, 
    criterion=criterion, 
    device=device, 
    partition_type=partition_type, 
    local_epochs=local_epochs,
    batch_size=batch_size,
    num_shards_per_partition=num_shards_per_partition,
    scheduler=scheduler,
)
server_app = get_server_app(
    model_class=model_class,
    model_config=model.get_config(), 
    optimizer=optimizer, 
    criterion=criterion, 
    device=device, 
    num_rounds=number_of_rounds, 
    min_available_clients=number_of_clients,
    min_fit_clients=min_num_clients,
    min_evaluate_clients=min_num_clients,
    checkpoint_dir=CHECKPOINT_DIR,
    fraction_fit=fraction_fit,
    fraction_evaluate=fraction_evaluate,
    #use_wandb=use_wandb,
    #wandb_config=wandb_config,
    scheduler=scheduler,
)

Using cache found in /home/massimiliano/.cache/torch/hub/facebookresearch_dino_main


Server on cuda
⚠️ No checkpoint found at /home/massimiliano/Projects/fl-g13/checkpoints. Creating a new model.


Using cache found in /home/massimiliano/.cache/torch/hub/facebookresearch_dino_main


In [5]:
run_simulation(
    client_app=client_app,
    server_app=server_app,
    num_supernodes=number_of_clients,
    backend_config=backend_config,
)

INFO :      Starting Flower ServerApp, config: num_rounds=2, no round_timeout
INFO :      
INFO :      [INIT]
INFO :      Using initial global parameters provided by strategy
INFO :      Starting evaluation of initial global parameters
Eval progress: 100%|██████████| 313/313 [00:23<00:00, 13.41batch/s]
INFO :      initial parameters (loss, other metrics): 6.5695034337881655, {'centralized_accuracy': 0.0107}
INFO :      
INFO :      [ROUND 1]
INFO :      configure_fit: strategy sampled 3 clients (out of 3)


[Round 0] Centralized Evaluation - Loss: 6.5695, Metrics: {'centralized_accuracy': 0.0107}


(ClientAppActor pid=134090) 2025-05-09 17:34:37.948 | INFO     | fl_g13.config:<module>:11 - PROJ_ROOT path is: /home/massimiliano/Projects/fl-g13


(ClientAppActor pid=134090) Client on cuda


Training progress:   0%|          | 0/209 [00:00<?, ?batch/s]


...

Eval progress: 100%|██████████| 53/53 [00:08<00:00,  6.24batch/s]


(ClientAppActor pid=134090) Client on cuda


Eval progress:   0%|          | 0/53 [00:00<?, ?batch/s]


(ClientAppActor pid=134090) Client on cuda


Eval progress: 100%|██████████| 53/53 [00:09<00:00,  5.83batch/s]
INFO :      aggregate_evaluate: received 3 results and 0 failures
INFO :      
INFO :      [SUMMARY]
INFO :      Run finished 2 round(s) in 584.35s
INFO :      	History (loss, distributed):
INFO :      		round 1: 1.202434407465113
INFO :      		round 2: 0.7252772566657396
INFO :      	History (loss, centralized):
INFO :      		round 0: 6.5695034337881655
INFO :      		round 1: 1.3155825675105135
INFO :      		round 2: 0.9102972100813168
INFO :      	History (metrics, distributed, fit):
INFO :      	{'avg_drift': [(1, 3.3214591896005246), (2, 2.2958337882619078)],
INFO :      	 'avg_train_loss': [(1, 4.159781643373523), (2, 2.0134234501183035)]}
INFO :      	History (metrics, distributed, evaluate):
INFO :      	{'federated_evaluate_accuracy': [(1, 0.6680663867226555),
INFO :      	                                 (2, 0.7752449510097981)]}
INFO :      	History (metrics, centralized):
INFO :      	{'centralized_accuracy': 